# Data Preprocessing:

Using NLTK

Steps: 
1. Import data
2. Extract features



In [7]:
import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import pandas as pd
import random
random.seed(32)

# close nltk download window to continue

## Import data

In [8]:
data = pd.read_csv("/Users/sophiasylvester/ma_py/pandora_bigfive1000.csv")

stopwordList = stopwords.words('english')

## Feature extraction

In [9]:
feature_df = data.filter(['author', 'body', 'word_count', 'word_count_quoteless', 'lang'])

# Total number of characters (including space)
feature_df['char_count'] = feature_df['body'].str.len()

# Total number of stopwords
feature_df['stopwords'] = feature_df['body'].apply(lambda x: len([x for x in x.split() if x in stopwordList]))

# Total number of punctuation or special characters
feature_df['total_punc'] = feature_df['body'].apply(lambda x: len([x for x in x.split() if x in string.punctuation]))
# should not be 0 in all cases

# Total number of numerics
feature_df['total_num'] = feature_df['body'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# Total number of uppercase words
feature_df['total_uppercase'] = feature_df['body'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

feature_df.head()

,author,body,word_count,word_count_quoteless,lang,char_count,stopwords,total_punc,total_num,total_uppercase
0,Sabata11792,Not seeing any break or signal lights and no p...,19,19,en,101,7,0,0,0
1,Swarels,"Multiverses, matrix theory, consciousness. Scr...",51,46,en,338,16,0,0,2
2,pearlz176,Hope you've enjoyed the ride :D,6,6,en,31,2,0,0,1
3,rainbowhotpocket,"Idk, in the AFC if i recall correctly since 20...",62,61,en,309,27,0,3,2
4,amathyx,22 hours later and the music is still going[co...,32,29,en,268,12,0,2,0


## Preprocessing

1. lower 
2. tokenize


In [10]:
comments = data.filter(['author', 'body'])
# comments.head()

for i in range(len(comments)):
    lowbody = comments.iloc[i,1].lower()
    lowbody = word_tokenize(comments.iloc[i,1])
    comments.iloc[i,1] = lowbody
    
comments.head()

,author,body
0,Sabata11792,"[Not, seeing, any, break, or, signal, lights, ..."
1,Swarels,"[Multiverses, ,, matrix, theory, ,, consciousn..."
2,pearlz176,"[Hope, you, 've, enjoyed, the, ride, :, D]"
3,rainbowhotpocket,"[Idk, ,, in, the, AFC, if, i, recall, correctl..."
4,amathyx,"[22, hours, later, and, the, music, is, still,..."
